<a href="https://colab.research.google.com/github/Cho-Yi-Ju/dorm_net_helper/blob/main/Bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [ ]:
import tensorflow
print(tensorflow.__version__)

1.15.2


In [ ]:
!pip install bert-serving-server
!pip install bert-serving-client
!pip install opencc-python-reimplemented

     |████████████████████████████████| 71kB 5.2MB/s 
  Created wheel for GPUtil: filename=GPUtil-1.4.0-cp36-none-any.whl size=7413 sha256=1bca541ee1d5e24313e2aabd466bbd86e551ebd9e549c46194f0059ea16cb4cb
  Stored in directory: /root/.cache/pip/wheels/3d/77/07/80562de4bb0786e5ea186911a2c831fdd0018bda69beab71fd
Successfully built GPUtil
     |████████████████████████████████| 491kB 9.5MB/s 
  Created wheel for opencc-python-reimplemented: filename=opencc_python_reimplemented-0.1.6-py2.py3-none-any.whl size=486136 sha256=1aefa5a446f181c9f748075fb1cd59d139981559de507b53cde90588aa0ddad6
  Stored in directory: /root/.cache/pip/wheels/54/8a/0f/405db0ba3f02d9af8211422d3ae94c47b3296256f168ac1e68
Successfully built opencc-python-reimplemented


In [ ]:
from opencc import OpenCC
from numpy import argmax
from scipy import spatial
from bert_serving.client import BertClient
from pandas import DataFrame, read_csv
import pandas as pd
import re
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.svm import SVC
from bert_serving.server.helper import get_args_parser
import sys
from bert_serving.server import BertServer
import os
from sklearn.metrics import accuracy_score

In [ ]:
if not os.path.exists('chinese_L-12_H-768_A-12/'):
  !unzip "drive/My Drive/dorm_net_helper/chinese_L-12_H-768_A-12.zip"

Archive:  drive/My Drive/dorm_net_helper/chinese_L-12_H-768_A-12.zip
   creating: chinese_L-12_H-768_A-12/
  inflating: chinese_L-12_H-768_A-12/bert_model.ckpt.meta  
  inflating: chinese_L-12_H-768_A-12/bert_model.ckpt.data-00000-of-00001  
  inflating: chinese_L-12_H-768_A-12/vocab.txt  
  inflating: chinese_L-12_H-768_A-12/bert_model.ckpt.index  
  inflating: chinese_L-12_H-768_A-12/bert_config.json  


In [ ]:
!unzip "drive/My Drive/dorm_net_helper/chinese_roberta_wwm_large_ext_L-24_H-1024_A-16.zip"

Archive:  drive/My Drive/dorm_net_helper/chinese_roberta_wwm_large_ext_L-24_H-1024_A-16.zip
  inflating: bert_config.json        
  inflating: bert_model.ckpt.data-00000-of-00001  
  inflating: bert_model.ckpt.index   
  inflating: bert_model.ckpt.meta    
  inflating: vocab.txt               


In [ ]:
PTHNAME="chinese_roberta_wwm_large_ext_L-24_H-1024_A-16"
!bert-serving-start -model_dir chinese_L-12_H-768_A-12/ -num_worker=1

In [ ]:
path = 'drive/My Drive/dorm_net_helper/'

In [ ]:
class Bert(object):
    def __init__(self, question):
        self.question = question

    def predict(self):
        df_a = pd.read_excel(path + "Train_QA.xlsx")
        punc = '[\W\d]'
        classes = []
        #sents_n_labels = []
        cc = OpenCC('t2s')
        for i in range(len(df_a['keyword'])):
            temp = df_a['keyword'][i]
            temp2 = re.sub(punc, ' ', temp.lower()).replace(" ", "")
            classes.append(temp2)
        answer = []

        for i in range(len(df_a['answer'])):
            temp = df_a['answer'][i]
            temp2 = temp.replace(" ", "")
            answer.append(temp2)
        labels = [int(t) for t in df_a['class']]
        classes = [cc.convert(s) for s in classes]
        bc = BertClient()
        classes_enc = bc.encode(classes)
        #print(classes_enc)
        sents = [cc.convert(self.question)]
        #sents = bc.encode(sents)
        aa = cosine_similarity(bc.encode(sents), classes_enc)
        #print(argmax(aa))
        #cls = SVC() #建立一個分類器
        #cls.fit(classes_enc, range(len(classes_enc)))
        #ans = cls.predict(sents)
        #return labels[ans[0]] , answer[ans[0]]
        if(aa[0][argmax(aa)] > 0.85):
            return labels[argmax(aa)] , answer[argmax(aa)]
        else:
            return 60 ,"不好意思，小幫手不了解你的意思"

In [ ]:
if __name__ == "__main__":
    args = get_args_parser().parse_args(['-model_dir', 'chinese_L-12_H-768_A-12/','-num_worker', '1'])
    server = BertServer(args)
    server.start()

    test = pd.read_excel(path + "Test.xlsx")
    punc = '[\W\d]'
    classes = []

    for i in range(len(test['keyword'])):
        temp = test['keyword'][i]
        temp2 = re.sub(punc, ' ', temp.lower()).replace(" ", "")
        classes.append(temp2)
    te_answer = []
    
    for i in range(len(test['answer'])):
        temp = test['answer'][i]
        temp2 = temp.replace(" ", "")
        te_answer.append(temp2)
    labels = [int(t) for t in test['class']]
    
    predict_classes = []
    predict_answers = []
    for c in classes:
        bert = Bert(c)
        predict_class , predict_ans = bert.predict()
        predict_classes.append(predict_class)
        predict_answers.append(predict_ans)
    print(accuracy_score(labels, predict_classes))
    '''
    for i in range(len(labels)):
        if(predict_classes[i] == labels[i]):
            print("True")
        else:
            print("False")
    '''
    #bert = Bert("要怎樣退錢")
    #predict_class = bert.predict()
    #print("predict: ", predict_class)


I:VENTILATOR:[__i:__i: 67]:freeze, optimize and export graph, could take a while...


I:GRAPHOPT:[gra:opt: 53]:model config: chinese_L-12_H-768_A-12/bert_config.json
I:GRAPHOPT:[gra:opt: 56]:checkpoint: chinese_L-12_H-768_A-12/bert_model.ckpt
I:GRAPHOPT:[gra:opt: 60]:build graph...
I:GRAPHOPT:[gra:opt:132]:load parameters from checkpoint...
I:GRAPHOPT:[gra:opt:136]:optimize...
I:GRAPHOPT:[gra:opt:144]:freeze...
I:GRAPHOPT:[gra:opt:149]:write graph to a tmp file: /tmp/tmpmal4z4e4
I:VENTILATOR:[__i:__i: 75]:optimized graph is stored at: /tmp/tmpmal4z4e4
I:VENTILATOR:[__i:_ru:129]:bind all sockets
I:VENTILATOR:[__i:_ru:180]:new config request	req id: 1	client: b'35177aeb-3ec9-40d4-bee2-4a5bd050e091'
I:VENTILATOR:[__i:_ru:196]:new encode request	req id: 2	size: 53	client: b'35177aeb-3ec9-40d4-bee2-4a5bd050e091'
I:VENTILATOR:[__i:_ru:196]:new encode request	req id: 3	size: 1	client: b'35177aeb-3ec9-40d4-bee2-4a5bd050e091'
I:VENTILATOR:[__i:_ru:180]:new config request	req id: 1	client: b'a41cd47e-35e0-4a71-a0e5-09a99f7395d3'
I:VENTILATOR:[__i:_ru:196]:new encode request	req i

0.6842105263157895
